In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import  torchvision
from torchvision import datasets as dsets
import torch.nn as nn
import torch.nn.functional as F
import os,time
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import itertools
import pickle
import imageio
import torch.optim as optim
from torch.autograd import Variable
import pdb

In [4]:
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [8]:
!unzip soccerballdata.zip

Archive:  soccerballdata.zip
   creating: soccerballdata/
   creating: soccerballdata/soccerball/
  inflating: soccerballdata/soccerball/image00019.jpg  
  inflating: soccerballdata/soccerball/image00021.jpg  
  inflating: soccerballdata/soccerball/image00022.jpg  
  inflating: soccerballdata/soccerball/image00023.jpg  
  inflating: soccerballdata/soccerball/image00039.jpg  
  inflating: soccerballdata/soccerball/image00042.jpg  
  inflating: soccerballdata/soccerball/image00046.jpg  
  inflating: soccerballdata/soccerball/image00057.jpg  
  inflating: soccerballdata/soccerball/image00067.jpg  
  inflating: soccerballdata/soccerball/image00070.jpg  
  inflating: soccerballdata/soccerball/image00098.jpg  
  inflating: soccerballdata/soccerball/image00134.jpg  
  inflating: soccerballdata/soccerball/image00173.jpg  
 extracting: soccerballdata/soccerball/image00176.jpg  
  inflating: soccerballdata/soccerball/image00183.jpg  
  inflating: soccerballdata/soccerball/image00217.jpg  
  infl

In [9]:
# Data augmentation and normalization for training
# Just normalization for validation
batch_size = 4
data_transforms = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])
data_dir = 'soccerballdata'
train_datasets = dsets.ImageFolder(data_dir,transform=data_transforms)
train_loaders = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size,
                                             shuffle=True, num_workers=4)

cuda = torch.cuda.is_available()
print(cuda)

True


In [10]:
len(train_datasets), train_datasets[0][0].shape, train_datasets[0][1]

(202, torch.Size([3, 64, 64]), 0)

In [0]:
def normal_init(m,mean,std):
  if isinstance(m, nn.ConvTranspose2d) or isinstance(m,nn.Conv2d):
    m.weight.data.normal_(mean,std)
    m.bias.data.zero_()
    
#weight_init  
def weight_init(self, mean=0.0, std=0.02):
  for m in self._modules:
    normal_init(self._modules[m], mean, std)

In [0]:
#Generator
# torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
#                          stride=1, padding=0, output_padding=0,
#                          groups=1, bias=True)
# output_height = (height-1)*stride + kernel_size - 2*padding + output_padding
# batch x 100-> batch x noisy input vector of size 100

class Generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 3, 4, 2, 1)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = F.relu(self.deconv1_bn(self.deconv1(input)))
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv5(x))

        return x

In [0]:
#Discriminator
class Discriminator(nn.Module):
  #initializers
  def __init__(self,d=128):
    super(Discriminator, self).__init__()
    self.conv1 = nn.Conv2d(3, d, 4, 2, 1)
    self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
    self.conv2_bn = nn.BatchNorm2d(d*2)
    self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
    self.conv3_bn = nn.BatchNorm2d(d*4)
    self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
    self.conv4_bn = nn.BatchNorm2d(d*8)
    self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)
    
  def forward(self,input):
    x = F.leaky_relu(self.conv1(input),0.2)
    x = F.leaky_relu(self.conv2_bn(self.conv2(x)),0.2)
    x = F.leaky_relu(self.conv3_bn(self.conv3(x)),0.2)
    x = F.leaky_relu(self.conv4_bn(self.conv4(x)),0.2)
    x = F.sigmoid(self.conv5(x))
    return x

In [39]:
fixed_z_ = torch.randn((5*5,100)).view(-1,100,1,1) # fixed noise for Generator
fixed_z_ = Variable(fixed_z_.cuda(), volatile = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [0]:
def show_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    z_ = torch.randn((5*5, 100)).view(-1, 100, 1, 1)
    z_ = Variable(z_.cuda(), volatile=True)

    G.eval()
    if isFix:
        test_images = G(fixed_z_)
    else:
        test_images = G(z_)
    G.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, 0].cpu().data.numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Iter')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [0]:
#training parameters
lr = 0.0002
train_epoch = 100
img_size = 64

In [42]:
#network
G = Generator()
D = Discriminator()

G.apply(weight_init)
D.apply(weight_init)

G.cuda(),D.cuda()

(Generator(
   (deconv1): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
   (deconv1_bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (deconv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
   (deconv2_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (deconv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
   (deconv3_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (deconv4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
   (deconv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (deconv5): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
 ), Discriminator(
   (conv1): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
   (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2,

In [43]:
#Binary Cross Entropy Loss
BCE_loss = nn.BCELoss()

#Adam Optimizers
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5,0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5,0.999))

G_optimizer,D_optimizer

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.5, 0.999)
     eps: 1e-08
     lr: 0.0002
     weight_decay: 0
 ), Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.5, 0.999)
     eps: 1e-08
     lr: 0.0002
     weight_decay: 0
 ))

In [0]:
# results save folder
if not os.path.isdir('Soccerball_DCGAN_results'):
    os.mkdir('Soccerball_DCGAN_results')
if not os.path.isdir('Soccerball_DCGAN_results/Random_results'):
    os.mkdir('Soccerball_DCGAN_results/Random_results')
if not os.path.isdir('Soccerball_DCGAN_results/Fixed_results'):
    os.mkdir('Soccerball_DCGAN_results/Fixed_results')

In [47]:
train_hist={}
train_hist['D_losses']=[]
train_hist['G_losses']=[]
train_hist['per_epoch_ptimes']=[]
train_hist['total_ptimes']=[]
num_iter=0

print('training start!')
start_time=time.time()
for epoch in range(train_epoch):
  D_losses = []
  G_losses = []
  epoch_start_time=time.time()
  for x_,label in train_loaders:
    #train Discriminator D
    D.zero_grad()
    mini_batch = x_.size()[0]
    y_real_ = torch.ones(mini_batch)
    y_fake_ = torch.zeros(mini_batch)
    
    x_,y_real_,y_fake_ = Variable(x_.cuda()), Variable(y_real_.cuda()), Variable(y_fake_.cuda())
    D_result = D(x_).squeeze()
    D_real_loss = BCE_loss(D_result, y_real_)
    
    z_ = torch.randn((mini_batch,100)).view(-1,100,1,1)
    z_ = Variable(z_.cuda())
    G_result = G(z_)
    D_result = D(G_result).squeeze()
    D_fake_loss = BCE_loss(D_result, y_fake_)
    D_fake_score = D_result.data.mean()
    
    D_train_loss = D_real_loss + D_fake_loss
    D_train_loss.backward()
    D_optimizer.step()
    
    # D_losses.append(D_train_loss.data[0])
    D_losses.append(D_train_loss.data[0])
    
    #train generator G
    G.zero_grad()
    z_ = torch.randn((mini_batch,100)).view(-1,100,1,1)
    z_ = Variable(z_.cuda())
    G_result = G(z_)
    D_result = D(G_result).squeeze()
    G_train_loss = BCE_loss(D_result, y_real_)
    G_train_loss.backward()
    G_optimizer.step()
    
    G_losses.append(G_train_loss.data[0])
    num_iter += 1
    
  epoch_end_time = time.time()
  per_epoch_ptimes = epoch_end_time - epoch_start_time
  
  print('[%d/%d] - ptime: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptimes, torch.mean(torch.FloatTensor(D_losses)),
                                                              torch.mean(torch.FloatTensor(G_losses))))
  p = 'Soccerball_DCGAN_results/Random_results/Soccerball_DCGAN_' + str(epoch + 1) + '.png'
  fixed_p = 'Soccerball_DCGAN_results/Fixed_results/Soccerball_DCGAN_' + str(epoch + 1) + '.png'
  show_result((epoch+1), save=True, path=p, isFix=False)
  show_result((epoch+1), save=True, path=fixed_p, isFix=True)
  train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
  train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))
  train_hist['per_epoch_ptimes'].append(per_epoch_ptimes)
  
end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptimes'].append(total_ptime)

print("Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_ptimes'])), train_epoch, total_ptime))
print("Training finish!... save training results")
torch.save(G.state_dict(), "Soccerball_DCGAN_results/generator_param.pkl")
torch.save(D.state_dict(), "Soccerball_DCGAN_results/discriminator_param.pkl")
with open('Soccerball_DCGAN_results/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

show_train_hist(train_hist, save=True, path='Soccerball_DCGAN_results/Soccerball_DCGAN_train_hist.png')

images = []
for e in range(train_epoch):
    img_name = 'Soccerball_DCGAN_results/Fixed_results/Soccerball_DCGAN_' + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('Soccerball_DCGAN_results/generation_animation.gif', images, fps=5)

training start!


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/100] - ptime: 12.68, loss_d: 0.889, loss_g: 12.632


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[2/100] - ptime: 13.30, loss_d: 0.659, loss_g: 8.871
[3/100] - ptime: 13.46, loss_d: 0.699, loss_g: 8.179
[4/100] - ptime: 13.50, loss_d: 0.476, loss_g: 6.769
[5/100] - ptime: 13.26, loss_d: 0.512, loss_g: 7.049
[6/100] - ptime: 13.29, loss_d: 0.662, loss_g: 6.250
[7/100] - ptime: 13.00, loss_d: 0.490, loss_g: 6.254
[8/100] - ptime: 12.98, loss_d: 0.580, loss_g: 6.839
[9/100] - ptime: 13.20, loss_d: 0.674, loss_g: 7.205
[10/100] - ptime: 13.15, loss_d: 0.680, loss_g: 5.534
[11/100] - ptime: 13.22, loss_d: 0.906, loss_g: 4.923
[12/100] - ptime: 13.27, loss_d: 0.825, loss_g: 3.935
[13/100] - ptime: 13.07, loss_d: 0.590, loss_g: 4.066
[14/100] - ptime: 13.32, loss_d: 0.712, loss_g: 3.885
[15/100] - ptime: 12.93, loss_d: 0.566, loss_g: 4.421
[16/100] - ptime: 13.18, loss_d: 0.587, loss_g: 4.226
[17/100] - ptime: 13.29, loss_d: 0.596, loss_g: 4.045
[18/100] - ptime: 13.46, loss_d: 0.527, loss_g: 4.292
[19/100] - ptime: 13.15, loss_d: 0.454, loss_g: 4.393
[20/100] - ptime: 13.00, loss_d: 0.3